# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [37]:
import os, requests
import random
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import numpy as np
import math

In [4]:
API_KEY = os.environ.get('key')

In [36]:
load_dotenv()
REGION = 'na1'
root = 'https://americas.api.riotgames.com'
winRateDict = {}
pickRateDict = {}
top5WR = {}

champListADC = ['aphelios', 'ashe', 'caitlyn', 'corki', 'draven', 'ezreal', 'jhin', 'jinx', 'kaisa', 'kalista', 'kogmaw', 'lucian', 'missfortune', 'nilah', 'samira', 'sivir', 'smolder', 'tristana', 'twitch', 'varus', 'vayne', 'xayah', 'zeri']
champListLengthADC = len(champListADC)
headersOPGG = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
}
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": API_KEY
}
topLaneWRDict = {}
topLanePRDict = {}

In [103]:
tierSort = '?tier=diamond_plus' #NOTE: Edit as needed

In [98]:
def updateWinRates(dictionary, headers, champList, tier, role):
    for champ in champList:
        opggURL = f'https://www.op.gg/champions/{champ}/build/{role}{tier}'
        response = requests.get(opggURL, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            winRateEm = soup.find('em', string='Win rate')
            if winRateEm:
                winRateB = winRateEm.find_next('b', class_='text-[12px]')
                if winRateB:
                    raw_text = winRateB.get_text(separator=" ", strip=True)
                    winRate = raw_text.replace('"', '').strip()
                    dictionary.update({champ: float(winRate.strip(" %"))})
            else:
                print(f"Win rate value not found for champion: {champ}")
        else:
            print(f"Win rate element not found for champion: {champ}")

def updatePickRates(dictionary, headers, champList, tier, role):
    for champ in champList:
        opggURL = f'https://www.op.gg/champions/{champ}/build/{role}{tier}'
        response = requests.get(opggURL, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            pickRateEm = soup.find('em', string='Pick rate')
            pickRateB = pickRateEm.find_next('b', class_='text-[12px]')
            if pickRateB:
                raw_text = pickRateB.get_text(separator=" ", strip=True)
                pickRate = raw_text.replace('"', '').strip()
                dictionary.update({champ: float(pickRate.strip(" %"))})
        else:
            print(f"Failed to retrieve data: {response.status_code}")

def getTop5(top5, winRateDict):
    for key, value in winRateDict.items():
        currWR = float(winRateDict.get(key))
        if len(top5) < 5:
            top5[key] = currWR
        else:
            smallest_key = min(top5, key=top5.get)
            if currWR > top5[smallest_key]:
                top5.pop(smallest_key)
                top5[key] = currWR

def addZScores(dictionary):
    zScoreDict = {}
    std = np.std(list(dictionary.values()))
    mean = np.mean(list(dictionary.values()))
    for key, value in dictionary.items():
        floatVal = float(value)
        currentKey = key
        zScore = float((floatVal - mean) / std)
        zScoreDict[key] = zScore
    return zScoreDict

def joinZPicKRate(winRateDict, zscoreDict):
    result = {}
    for key, value in winRateDict.items():
        result.update({key: 1.5 * value + 1.7 * (zscoreDict[key])})
    return result


def getMatchHistory(count, username, tagline, headers):
    matches = requests.get(f'{root}/lol/match/v5/matches/by-puuid/{playerPUUID}/ids?start=0&count={count}&api_key={API_KEY}', headers=headers).json()
    return matches

def getMatchInformation(matchID, headers):
    return requests.get(f'{root}/lol/match/v5/matches/{matchID}?api_key={API_KEY}', headers=headers).json()

In [48]:
def getChampsFromOPGG(headersOPGG, role):
    url = f"https://www.op.gg/champions?position={role}"
    response = requests.get(url, headers=headersOPGG)
    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
    
    champList = []
    for tag in soup.find_all('strong', class_='flex-1 truncate text-xs max-[420px]:sr-only'):
        name = tag.get_text(strip=True).lower()
        clean_name = name.replace("'", "").replace(".", "").replace(" ", "")
        champList.append(clean_name)
    return champList

In [60]:
champListTOP = getChampsFromOPGG(headersOPGG, "top")
champListTOP.remove("swain")
champListTOP.remove("tahmkench")
champListTOP.remove("wukong")
print(champListTOP)

['riven', 'quinn', 'kayle', 'yorick', 'sett', 'darius', 'gwen', 'teemo', 'gragas', 'fiora', 'garen', 'warwick', 'malphite', 'camille', 'irelia', 'aatrox', 'jax', 'kennen', 'gangplank', 'urgot', 'gnar', 'yone', 'chogath', 'jayce', 'renekton', 'singed', 'sion', 'shen', 'nasus', 'vladimir', 'ambessa', 'olaf', 'ornn', 'pantheon', 'tryndamere', 'drmundo', 'yasuo', 'poppy', 'volibear', 'mordekaiser', 'cassiopeia', 'kled', 'ryze', 'trundle', 'vayne', 'illaoi', 'heimerdinger', 'rumble', 'ksante', 'sylas', 'akali', 'zac']


In [104]:
updateWinRates(topLaneWRDict, headersOPGG, champListTOP, tierSort, "top")
print(topLaneWRDict)

{'riven': 53.53, 'quinn': 53.54, 'kayle': 51.66, 'yorick': 50.8, 'sett': 52.0, 'darius': 50.02, 'fiora': 52.93, 'warwick': 52.73, 'gwen': 51.21, 'teemo': 49.55, 'garen': 48.19, 'gragas': 52.4, 'camille': 50.97, 'malphite': 51.14, 'aatrox': 49.2, 'irelia': 51.96, 'jax': 50.8, 'kennen': 53.97, 'gangplank': 50.27, 'urgot': 50.25, 'yone': 50.85, 'gnar': 50.36, 'singed': 50.55, 'jayce': 50.17, 'renekton': 48.76, 'sion': 51.73, 'shen': 50.6, 'nasus': 52.02, 'vladimir': 51.75, 'ambessa': 49.58, 'olaf': 52.2, 'pantheon': 51.39, 'ornn': 48.73, 'yasuo': 49.02, 'tryndamere': 51.39, 'volibear': 48.77, 'poppy': 50.9, 'mordekaiser': 47.66, 'ryze': 51.19, 'cassiopeia': 51.97, 'kled': 51.62, 'trundle': 49.54, 'illaoi': 49.25, 'vayne': 51.62, 'heimerdinger': 50.9, 'rumble': 48.75, 'sylas': 48.36, 'akali': 46.08, 'zac': 48.18, 'chogath': 50.66, 'drmundo': 50.26, 'ksante': 47.76}


In [105]:
updatePickRates(topLanePRDict, headersOPGG, champListTOP, tierSort, "top")
print(topLanePRDict)

{'riven': 5.49, 'quinn': 1.64, 'kayle': 3.19, 'yorick': 4.04, 'sett': 5.21, 'darius': 6.2, 'gwen': 4.91, 'teemo': 2.12, 'gragas': 3.61, 'fiora': 4.92, 'garen': 8.5, 'warwick': 2.21, 'malphite': 4.03, 'camille': 4.54, 'irelia': 4.18, 'aatrox': 10.05, 'jax': 5.35, 'kennen': 2.01, 'gangplank': 4.19, 'urgot': 3.28, 'gnar': 4.49, 'yone': 5.7, 'chogath': 2.58, 'jayce': 7.71, 'renekton': 5.11, 'singed': 2.44, 'sion': 3.71, 'shen': 3.72, 'nasus': 1.94, 'vladimir': 3.17, 'ambessa': 5.89, 'olaf': 2.3, 'ornn': 3.51, 'pantheon': 2.43, 'tryndamere': 2.29, 'drmundo': 2.59, 'yasuo': 3.04, 'poppy': 1.57, 'volibear': 3.41, 'mordekaiser': 3.73, 'cassiopeia': 0.96, 'kled': 1.26, 'ryze': 1.42, 'trundle': 1.33, 'vayne': 1.08, 'illaoi': 1.36, 'heimerdinger': 0.83, 'rumble': 2.33, 'ksante': 3.9, 'sylas': 1.64, 'akali': 1.57, 'zac': 0.93}


In [106]:
zScorePRDictTOP = addZScores(topLanePRDict)
zScoreWRDictTOP = addZScores(topLaneWRDict)
truePowerLevelDictTOP = joinZPicKRate(zScorePRDictTOP, zScoreWRDictTOP)

In [107]:
top5PLTOP = dict(sorted(truePowerLevelDictTOP.items(), key=lambda item: item[1], reverse=True)[:5])
truePowerLevelDictTOP = dict(sorted(truePowerLevelDictTOP.items(), key=lambda item: item[1], reverse=True))
sortedWRTOP = dict(sorted(topLaneWRDict.items(), key=lambda item: item[1], reverse=True))
sortedPRTOP = dict(sorted(topLanePRDict.items(), key=lambda item: item[1], reverse=True))

print(f'Win Rates: {sortedWRTOP}\n')
print(f'Pick Rates: {sortedPRTOP}\n')
print(f'Power Level: {truePowerLevelDictTOP}\n')
print(f'Top 5: {top5PLTOP}\n')

Win Rates: {'kennen': 53.97, 'quinn': 53.54, 'riven': 53.53, 'fiora': 52.93, 'warwick': 52.73, 'gragas': 52.4, 'olaf': 52.2, 'nasus': 52.02, 'sett': 52.0, 'cassiopeia': 51.97, 'irelia': 51.96, 'vladimir': 51.75, 'sion': 51.73, 'kayle': 51.66, 'kled': 51.62, 'vayne': 51.62, 'pantheon': 51.39, 'tryndamere': 51.39, 'gwen': 51.21, 'ryze': 51.19, 'malphite': 51.14, 'camille': 50.97, 'poppy': 50.9, 'heimerdinger': 50.9, 'yone': 50.85, 'yorick': 50.8, 'jax': 50.8, 'chogath': 50.66, 'shen': 50.6, 'singed': 50.55, 'gnar': 50.36, 'gangplank': 50.27, 'drmundo': 50.26, 'urgot': 50.25, 'jayce': 50.17, 'darius': 50.02, 'ambessa': 49.58, 'teemo': 49.55, 'trundle': 49.54, 'illaoi': 49.25, 'aatrox': 49.2, 'yasuo': 49.02, 'volibear': 48.77, 'renekton': 48.76, 'rumble': 48.75, 'ornn': 48.73, 'sylas': 48.36, 'garen': 48.19, 'zac': 48.18, 'ksante': 47.76, 'mordekaiser': 47.66, 'akali': 46.08}

Pick Rates: {'aatrox': 10.05, 'garen': 8.5, 'jayce': 7.71, 'darius': 6.2, 'ambessa': 5.89, 'yone': 5.7, 'riven': 5

In [45]:
updateWinRates(winRateDict, headersOPGG, champListADC)
updatePickRates(pickRateDict, headersOPGG, champListADC)

In [46]:
getTop5(top5WR, winRateDict)

In [47]:
getTop5(top5WR, winRateDict)

In [48]:
zScorePRDict = addZScores(pickRateDict)
zScoreWRDict = addZScores(winRateDict)

In [49]:
truePowerLevelDict = joinZPicKRate(zScoreWRDict, zScorePRDict)

top5PL = dict(sorted(truePowerLevelDict.items(), key=lambda item: item[1], reverse=True)[:5])
truePowerLevelDict = dict(sorted(truePowerLevelDict.items(), key=lambda item: item[1], reverse=True))
sortedWR = dict(sorted(winRateDict.items(), key=lambda item: item[1], reverse=True))
sortedPR = dict(sorted(pickRateDict.items(), key=lambda item: item[1], reverse=True))

print(f'Win Rates: {sortedWR}\n')
print(f'Pick Rates: {sortedPR}\n')
print(f'Power Level: {truePowerLevelDict}\n')
print(f'Top 5: {top5PL}\n')

Win Rates: {'nilah': 53.31, 'kogmaw': 53.27, 'tristana': 52.15, 'jinx': 51.91, 'draven': 51.73, 'vayne': 51.53, 'missfortune': 51.47, 'corki': 51.37, 'lucian': 51.06, 'ashe': 50.95, 'twitch': 50.63, 'varus': 50.56, 'xayah': 50.51, 'zeri': 50.44, 'aphelios': 50.27, 'kaisa': 50.23, 'samira': 50.02, 'jhin': 49.85, 'smolder': 49.74, 'sivir': 49.7, 'caitlyn': 49.22, 'kalista': 49.03, 'ezreal': 48.47}

Pick Rates: {'kaisa': 24.43, 'ezreal': 20.5, 'jhin': 16.79, 'tristana': 13.74, 'jinx': 13.41, 'varus': 10.59, 'lucian': 10.04, 'ashe': 7.9, 'caitlyn': 7.41, 'corki': 6.54, 'aphelios': 6.09, 'missfortune': 5.75, 'draven': 5.54, 'xayah': 5.05, 'smolder': 4.33, 'zeri': 4.18, 'twitch': 3.82, 'sivir': 3.63, 'vayne': 3.57, 'kalista': 2.63, 'samira': 2.6, 'kogmaw': 2.03, 'nilah': 1.93}

Power Level: {'kaisa': 4.879181901587604, 'tristana': 3.679156743102831, 'jinx': 3.270064305873314, 'jhin': 1.8432374725210499, 'ezreal': 1.3725034310934525, 'nilah': 1.1585066074679782, 'kogmaw': 1.1423334259489621, 

In [50]:
try:
    username, tagline = input("Enter username and tagline (e.g. Saico#Slide): ").split("#")
    print(f"Username: {username}, Tagline: {tagline}")
except ValueError:
    print("Invalid entry method, try again.\n")

Username: Saico, Tagline: Slide


In [56]:
def getPUUID(username, tagline, headers):
    return requests.get(f'{root}/riot/account/v1/accounts/by-riot-id/{username}/{tagline}?api_key={API_KEY}', headers=headers).json()['puuid']

In [57]:
playerPUUID = getPUUID(username, tagline, headers)
count = 20

matchList = getMatchHistory(count, username, tagline)
print(matchList)

['NA1_5257793042', 'NA1_5257769139', 'NA1_5257751614', 'NA1_5257733241', 'NA1_5257661308', 'NA1_5257625478', 'NA1_5257614615', 'NA1_5257304145', 'NA1_5257282077', 'NA1_5257187868', 'NA1_5257170379', 'NA1_5257158710', 'NA1_5257147580', 'NA1_5256858062', 'NA1_5256837726', 'NA1_5256830246', 'NA1_5256819084', 'NA1_5255639416', 'NA1_5255620896', 'NA1_5255598057']


In [ ]:
#Give each piece of information a rating from -100-100 in terms of importance 1 is least important 100 is most important
for game in matchList:
    matchInfo = getMatchInformation(game)
    info = matchInfo['info']
    queueID = info['queueId'] #Ranked is queueID 420
    if queueID != 420:
        continue
    metadata = matchInfo['metadata']
    players = info['participants']
    participants = metadata['participants']
    teams = info['teams']
    player = players[participants.index(playerPUUID)]
    playerTeam = player['teamId']
    matchID = metadata['matchId']
    gameStartTime = info['gameStartTimestamp'] #???
    gameEndTime = info['gameEndTimestamp'] #???
    for team in teams:
        if team['teamId'] == playerTeam:
            objs = team['objectives'] #See how its printed before assigning rank
            baron = objs['baron'] #85 * numBarons
            dragon = objs['dragon'] #20 * numDrags
            # grubs = objs['grubs'] Not available :(
            herald = objs['riftHerald'] #55-60
            tower = objs['tower'] #8.63 * numTurrets
            inhibitor = objs['inhibitor'] #Depends on how many down at a time and end time relative to when first inhibitor was taken
            champion = objs['champion'] #???
    for obj in [baron, dragon, herald, tower, inhibitor]:
        first = obj['first']
        objKills = obj['kills'] #(4 Drags + 30)
    assists = player['assists'] #3*numDeaths
    champLevel = player['champLevel'] #
    champName = player['championName'] #
    damageToBuildings = player['damageDealtToBuildings'] #
    deaths = player['deaths'] #10*numDeaths
    doublekills = player['doubleKills'] #
    firstBloodAssist = player['firstBloodAssist'] #9
    firstBloodKill = player['firstBloodKill'] #12
    firstTowerAssist = player['firstTowerAssist'] #20 
    firstTowerKill = player['firstTowerKill'] #20
    gold = player['goldEarned'] 
    kills = player['kills'] #6*numKills
    longestTimeLiving = player['longestTimeSpentLiving']
    neutralMinionsKilled = player['neutralMinionsKilled']
    minionsKilled = player['totalMinionsKilled']
    csScore = minionsKilled + 4*neutralMinionsKilled #1.11 * csScore hard to evaluate if also trying to do this for support
    turretTakedowns = player['turretTakedowns'] #
    visionScore = player['visionScore']
    win = player['win']
    wardsPlaced = player['wardsPlaced']
    wardsKilled = player['wardsKilled']
    ctrlWardsBought = player['visionWardsBoughtInGame']